In [ ]:
#basic library
import pandas
import matplotlib.pyplot as plt
import numpy as np

# Install tokenize
!pip install pyvi
from pyvi import ViTokenizer

# import necessary libraries for deep learning
import warnings
import keras
warnings.filterwarnings("ignore")
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

In [ ]:
#load data
#file output.csv nay sinh ra tu chuong trinh scraping
data_path=r'/content/drive/MyDrive/Học tập/Introduce Data Science/Đố án/output.csv'
df= pandas.read_csv(data_path)

#preprocessing
df = df[pandas.notnull(df['postCotent'])]
df= df[df['forumName'] != 'Chuyện trò linh tinh™'] 
df=df[df['forumName'].map(len) >2]

labels=set(df['forumName'])

data=dict()
#Get labels
for label in labels:
  data[label]=[]
for index,row in df.iterrows():
  for label in labels:
    if row['forumName']==label:
      data[label].append(row['postCotent'])

for label in labels:
  print(label,': ',len(data[label]))

#convert label to num
def labelToNum(label):
  for (index, value) in enumerate(labels):
    if value==label:
      return index;
  return 0;
def numToLabel(num):
  for (index, value) in enumerate(labels):
    if index==num:
      return value;
  return data[data.keys()[0]];


df.head()

In [ ]:
# Data to plot
labels=labels
countings = []
for label in labels:
  countings.append(len(data[label]))

# Plot
plt.pie(countings, labels=labels,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()

In [76]:
X=list(df['postCotent'])
Y=list(df['forumName'])
Y=[labelToNum(i) for i in Y]

print(X[:10])
print(Y[:10])
print(Y[:10])

['Máy ngoại hình tốt, ko sướt cấn móp. Pin 100%. Máy đã dán ppf, ốp, kính cường lực.\nGiấy tờ hóa đơn mua còn đầy đủ. Trong tgian bảo hành còn được thay pin miễn phí. \nMáy đang dùng song song với XR, dư nên bán. Sdt liên hệ 0908 607 675.\nGiá chốt 7.400.000 vnđ\n. ', 'Hello mấy thím, nhân hôm không phải trông con giúp vợ, được dịp rảnh rỗi chém gió tí về mấy món đồ công nghệ mình sưu tầm được. Tôi là dân cơ khí nên mê mấy con gia dụng công nghệ vãi chưởng. Mà bị cái, mỗi lần tôi tha 1 món về thì vợ tôi cứ ầm ĩ bên tai. Nhà tôi thì không quá dư dả gì nhưng có con máy nào ra mà tốt thì tôi cũng ráng đi nghía rồi để tiền mua. Tổng cộng tôi đã tậu đc sương sương vài em như robot hút bụi Neato d5, máy rửa chén tự động Xiaomi Viomi, máy lọc không khí Beurer. Nhìn chung thì mấy món tôi mua giá cũng ko quá cao nhưng mà xài cũng đc. Đợt black friday trước, tôi quyết định chơi sang tự vác tiền mua con máy giặt của LG tên AI DD.\nĐợt tôi mua là cũng gan trời lắm vì vợ nhà không biết, tới khi biế

In [ ]:
def tokenize(text):
  return ViTokenizer.tokenize(text).split()

data=[]
for i in X:
  data.append(tokenize(i))

data[0]

In [78]:
word_tokenizer = Tokenizer() 
word_tokenizer.fit_on_texts(data)  

X= word_tokenizer.texts_to_sequences(X)  # use the tokeniser to encode input sequence

In [79]:
MAX_SEQ_LENGTH = 100
X_padded = pad_sequences(X, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")

In [80]:
EMBEDDING_SIZE  = 300  # each word in fast model is represented using a 300 dimensional vector
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1

#link dowload: https://thiaisotajppub.s3-ap-northeast-1.amazonaws.com/publicfiles/baomoi.window2.vn.model.bin.gz
path = '/content/drive/MyDrive/Học tập/Introduce Data Science/word2ve/baomoi.window2.vn.model.bin'
word2vec = KeyedVectors.load_word2vec_format(path, binary=True)


EMBEDDING_SIZE  = 300  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1

# create an empty embedding matix
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))

# create a word to index dictionary mapping
word2id = word_tokenizer.word_index

# copy vectors from word2vec model to the words present in corpus
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

In [81]:
Y= to_categorical(Y)

In [82]:
rnn_model = Sequential()

# create embedding layer - usually the first layer in text problems
rnn_model.add(Embedding(input_dim     =  VOCABULARY_SIZE,         # vocabulary size - number of unique words in data
                        output_dim    =  EMBEDDING_SIZE,          # length of vector with which each word is represented
                        input_length  =  MAX_SEQ_LENGTH,          # length of input sequence
                        weights       = [embedding_weights],      # word embedding matrix
                        trainable     =  True                     # True - update the embeddings while training
))

# add an RNN layer which contains 64 RNN cells
rnn_model.add(Bidirectional(LSTM(64, 
              return_sequences=False  # True - return whole sequence; False - return single output of the end of the sequence
)))

rnn_model.add(Dense(4, activation='softmax'))

In [83]:
rnn_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          9052500   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 9,239,896
Trainable params: 9,239,896
Non-trainable params: 0
_________________________________________________________________


In [84]:
rnn_model.compile(loss      =  'categorical_crossentropy',
                  optimizer =  'adam',
                  metrics   =  ['acc'])

In [85]:
rnn_training = rnn_model.fit(X_padded, Y, batch_size=128, epochs=10)

Epoch 1/10
42/42 [==============================] - 29s 626ms/step - loss: 1.2039 - acc: 0.4740
Epoch 2/10
42/42 [==============================] - 27s 632ms/step - loss: 0.4914 - acc: 0.8322
Epoch 3/10
42/42 [==============================] - 27s 633ms/step - loss: 0.2480 - acc: 0.9239
Epoch 4/10
42/42 [==============================] - 27s 633ms/step - loss: 0.1546 - acc: 0.9504
Epoch 5/10
42/42 [==============================] - 27s 632ms/step - loss: 0.0940 - acc: 0.9746
Epoch 6/10
42/42 [==============================] - 27s 635ms/step - loss: 0.0698 - acc: 0.9836
Epoch 7/10
42/42 [==============================] - 27s 639ms/step - loss: 0.0686 - acc: 0.9832
Epoch 8/10
42/42 [==============================] - 27s 643ms/step - loss: 0.0425 - acc: 0.9896
Epoch 9/10
42/42 [==============================] - 27s 640ms/step - loss: 0.0536 - acc: 0.9883
Epoch 10/10
42/42 [==============================] - 30s 707ms/step - loss: 0.0529 - acc: 0.9845


In [86]:
#chon thu muc can luu
rnn_model.save('/content/drive/MyDrive/Học tập/Introduce Data Science/model.h5')

In [87]:
test='Điện thoại Iphone10 vừa ra mắt thật đẹp muốn mua mà không có tiền'

def embedding(text):
  text= ViTokenizer.tokenize(text).split(text)
  
  text = word_tokenizer.texts_to_sequences(text)
  
  
  text= pad_sequences(text, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")

  return text;

test=embedding(test)

rs=rnn_model.predict(test)
rs
#ket qua => xac suat thuoc lop 1 la 0.86 => thuoc lop 1

array([[0.03137342, 0.8692496 , 0.00534497, 0.09403204]], dtype=float32)

In [89]:
# lay nhan cua lop 1
numToLabel(1)

'Điện thoại di động'